In [1]:
import requests
from pyquery import PyQuery as pq
#import json
import csv,codecs#解决乱码！
import os 
import numpy as np
from hashlib import md5
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.zhihu.com/question/438588361/answer/1703903763'#'https://www.zhihu.com/question/437319323/answer/1785586165'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

In [6]:
base_url = 'https://www.zhihu.com/api/v4/questions/421463194/answers?'
include = 'data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cattachment%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Cis_labeled%2Cpaid_info%2Cpaid_info_content%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_recognized%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics%3Bdata%5B%2A%5D.settings.table_of_content.enabled'
#https://www.zhihu.com/api/v4/questions/438588361/answers?
#include=data%5B*%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cattachment%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Cis_labeled%2Cpaid_info%2Cpaid_info_content%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_recognized%3Bdata%5B*%5D.mark_infos%5B*%5D.url%3Bdata%5B*%5D.author.follower_count%2Cvip_info%2Cbadge%5B*%5D.topics%3Bdata%5B*%5D.settings.table_of_content.enabled
#&offset=&limit=3&sort_by=default&platform=desktop

def get_page(page):#page0就是第一页
    url1 = 'include=' + include+ '&limit=5&' + 'offset=' + str(page)+ '&platform=desktop&sort_by=default'
    url = base_url + url1#urlencode(params)
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
    except requests.ConnectionError as e:
        print('Error', e.args)

In [7]:
def parse_page(json):
    if json:
        items = json.get('data')
        for item in items:
            zhihu = {}
            zhihu['作者'] = item.get('author').get('name')
            zhihu['回答'] = pq(item.get('content')).text()
            zhihu['赞'] = item.get('voteup_count')
            yield zhihu#生成器

In [8]:
if __name__=='__main__':
    i = 0
    f = codecs.open('对于笔记本的选择，轻薄本真的被看不起吗？.csv', 'w+', 'utf_8_sig')
    ftxt = open('对于笔记本的选择，轻薄本真的被看不起吗？.txt', 'w+', encoding='utf_8')
    fieldnames = ['作者', '回答','赞']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    while True:
        js = get_page(5*i)
        results = parse_page(js)
        for res in results:
            writer.writerow(res)
            for detail in res.values():
                ftxt.write(str(detail)+'\n')
            ftxt.write('\n' + '=' * 50 + '\n')
        if js.get('paging').get('is_end'):
            print('finish!')
            break
        i+=1
    f.close()
    ftxt.close()

ParserError: Document is empty